In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Function to load the violence dataset (you need to adjust the path)
def load_violence_dataset(path):
    videos = []
    labels = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        print(f"Processing folder: {folder_path}")
        if os.path.isdir(folder_path):
            label = 1 if 'violence' in folder.lower() else 0  # 1: Violence, 0: Non-violence
            for file in os.listdir(folder_path):
                if file.endswith(('.jpg', '.png')):
                    image_path = os.path.join(folder_path, file)
                    try:
                        image = cv2.imread(image_path)
                        if image is None:
                            print(f"Failed to load image {image_path}. Skipping...")
                            continue

                        image = cv2.resize(image, (224, 224))  # Resize to the model input size
                        videos.append(image)
                        labels.append(label)
                    except Exception as e:
                        print(f"Error processing image {image_path}: {e}")
                        continue
    videos = np.array(videos)
    labels = np.array(labels)
    return videos, labels

# Preprocess the dataset
def preprocess_data(images):
    images = images.astype('float32') / 255.0
    return images

# Autoencoder Model
def create_autoencoder(input_shape):
    # Encoder
    input_img = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    # Bottleneck
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    
    # Decoder
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    return autoencoder

# Compile the model
def compile_autoencoder(autoencoder):
    autoencoder.compile(optimizer=Adam(learning_rate=0.0001), 
                        loss='mean_squared_error',
                        metrics=['accuracy'])

# Train the autoencoder model
def train_autoencoder(autoencoder, train_images, val_images):
    batch_size = 8
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = autoencoder.fit(train_images, train_images,  # Autoencoders train to reconstruct the input
                              validation_data=(val_images, val_images),
                              epochs=15,
                              batch_size=batch_size,
                              callbacks=[early_stopping])
    return history

# Test the model and calculate performance metrics
def test_autoencoder(autoencoder, test_images, true_labels, threshold=0.02):
    reconstructions = autoencoder.predict(test_images)
    
    # Compute reconstruction error
    errors = np.mean(np.abs(reconstructions - test_images), axis=(1, 2, 3))
    anomalies = errors > threshold
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, anomalies)
    precision = precision_score(true_labels, anomalies)
    recall = recall_score(true_labels, anomalies)
    f1 = f1_score(true_labels, anomalies)
    
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    
    return errors, anomalies, accuracy, precision, recall, f1

# Main function to execute the workflow
def main():
    dataset_path = r'C:\Users\abhishik chebrolu\Downloads\AINN pro\archive\new_violence'  # Update with the actual dataset path
    
    # Load the dataset and labels
    images, labels = load_violence_dataset(dataset_path)
    images = preprocess_data(images)
    
    # Split into training and testing sets (80% training, 20% testing)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42, stratify=labels)
    
    # Further split training data into training and validation sets (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)
    
    # Create and compile the autoencoder model
    input_shape = (224, 224, 3)
    autoencoder = create_autoencoder(input_shape)
    compile_autoencoder(autoencoder)
    
    # Train the autoencoder
    print("Training the autoencoder...")
    history = train_autoencoder(autoencoder, X_train, X_val)
    
    # Plot training history
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()
    
    # Test the autoencoder and calculate accuracy, precision, recall, F1
    print("Testing the autoencoder...")
    errors, anomalies, accuracy, precision, recall, f1 = test_autoencoder(autoencoder, X_test, y_test)

if __name__ == "__main__":
    main()

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Create Transfer Learning Autoencoder (VGG16 as Encoder)
def create_transfer_learning_autoencoder(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze the encoder (VGG16) layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Decoder (ensure correct output size: 224x224x3)
    x = base_model.output  # The output from VGG16 will be (7, 7, 512)

    # Decoder starts here: reverse the pooling/downsampling
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)  # From (7, 7, 512) to (14, 14, 512)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)  # From (14, 14, 256) to (28, 28, 256)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)  # From (28, 28, 128) to (56, 56, 128)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)  # From (56, 56, 64) to (112, 112, 64)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)  # From (112, 112, 32) to (224, 224, 32)

    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)  # Final output shape (224, 224, 3)

    # Create autoencoder model
    autoencoder = Model(base_model.input, decoded)
    return autoencoder

# Compile the model
def compile_autoencoder(autoencoder):
    autoencoder.compile(optimizer=Adam(learning_rate=0.0001), 
                        loss='mean_squared_error',
                        metrics=['accuracy'])

# Test the model and calculate performance metrics
def test_autoencoder(autoencoder, test_generator):
    reconstructions = []
    true_labels = []

    # Batch-wise reconstruction and error calculation
    for batch in test_generator:
        reconstructed_batch = autoencoder.predict(batch)
        reconstructions.append(reconstructed_batch)
        true_labels.append(batch)  # The input batch itself is used as the ground truth

    # Convert lists to numpy arrays
    reconstructions = np.concatenate(reconstructions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)

    # Compute reconstruction error (Mean Squared Error) for each sample
    errors = np.mean(np.abs(reconstructions - true_labels), axis=(1, 2, 3))

    # Use a threshold to classify anomalies (assuming high reconstruction error as anomaly)
    threshold = 0.02  # You can tune this value based on the dataset
    anomalies = errors > threshold

    # Test labels (for binary classification task)
    filenames = test_generator.filenames
    true_class_labels = np.array([1 if 'violence' in filename else 0 for filename in filenames])

    # Calculate metrics
    accuracy = accuracy_score(true_class_labels, anomalies)
    precision = precision_score(true_class_labels, anomalies)
    recall = recall_score(true_class_labels, anomalies)
    f1 = f1_score(true_class_labels, anomalies)

    print(f"Testing Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

    # Confusion matrix
    cm = confusion_matrix(true_class_labels, anomalies)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return accuracy, precision, recall, f1

# Main function to execute workflow
def main():
    dataset_path = r'C:\Users\abhishik chebrolu\Downloads\AINN pro\archive\new_violence'  # Path to your dataset with violence and non-violence frames
    input_shape = (224, 224, 3)
    
    # Create and compile the autoencoder
    autoencoder = create_transfer_learning_autoencoder(input_shape)
    compile_autoencoder(autoencoder)
    
    # Use ImageDataGenerator for data loading and augmentation
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        validation_split=0.3  # Split 70-30 for training and testing
    )
    
    batch_size = 32
    
    # Train and validation data generators
    train_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='input',  # For autoencoder, input and target are the same
        subset='training'
    )
    
    # Test data generator (30% test split, no augmentation)
    test_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='input',  # For autoencoder, input and target are the same
        subset='validation',
        shuffle=False
    )
    
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the autoencoder
    history = autoencoder.fit(
        train_generator,
        validation_data=test_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_steps=test_generator.samples // batch_size,
        epochs=10,
        callbacks=[early_stopping]
    )
    
    # Testing accuracy
    print("Testing the autoencoder...")
    test_accuracy, precision, recall, f1 = test_autoencoder(autoencoder, test_generator)

    # Plot training accuracy and testing accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.axhline(y=test_accuracy, color='r', linestyle='--', label='Testing Accuracy')
    plt.title('Training and Testing Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


Found 7752 images belonging to 2 classes.
Found 3321 images belonging to 2 classes.
Epoch 1/10


C:\Users\abhishik chebrolu\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


242/242 ━━━━━━━━━━━━━━━━━━━━ 627s 3s/step - accuracy: 0.4533 - loss: 0.0633 - val_accuracy: 0.5443 - val_loss: 0.0444
Epoch 2/10
  1/242 ━━━━━━━━━━━━━━━━━━━━ 7:21 2s/step - accuracy: 0.5656 - loss: 0.0374

c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


242/242 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5656 - loss: 0.0374 - val_accuracy: 0.4266 - val_loss: 0.0492
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 631s 3s/step - accuracy: 0.5057 - loss: 0.0401 - val_accuracy: 0.5622 - val_loss: 0.0407
Epoch 4/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5323 - loss: 0.0396 - val_accuracy: 0.4116 - val_loss: 0.0448
Epoch 5/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 729s 3s/step - accuracy: 0.5320 - loss: 0.0357 - val_accuracy: 0.5272 - val_loss: 0.0380
Epoch 6/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - accuracy: 0.5785 - loss: 0.0393 - val_accuracy: 0.4990 - val_loss: 0.0399
Epoch 7/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 940s 4s/step - accuracy: 0.5433 - loss: 0.0335 - val_accuracy: 0.5767 - val_loss: 0.0358
Epoch 8/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5678 - loss: 0.0319 - val_accuracy: 0.5016 - val_loss: 0.0417
Epoch 9/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 715s 3s/step - accuracy: 0.5575 - loss: 0.0314 - val_accuracy: 0.5777 

ValueError: Layer "functional" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(32, 224, 224, 3) dtype=float32>, <tf.Tensor 'data_1:0' shape=(32, 224, 224, 3) dtype=float32>]